In [2]:
from iwatlas import sshdriver
from iwatlas import harmonics
from iwatlas import stratification as strat

from sfoda.utils.mynumpy import grad_z

import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

from scipy.optimize import least_squares
from tqdm import tqdm

import matplotlib.pyplot as plt

In [3]:
basedir = '/home/suntans/cloudstor/Data/IWAtlas'
sshfile = '{}/NWS_2km_GLORYS_hex_2013_2014_SSHBC_Harmonics.nc'.format(basedir)
ampfile = '{}/NWS_2km_GLORYS_hex_2013_2014_Amplitude_Atlas.nc'.format(basedir)
climfile = '{}/NWS_2km_GLORYS_hex_2013_2014_Climatology.nc'.format(basedir)

# Output filename
N2file = '{}/NWS_2km_GLORYS_hex_2013_2014_Stratification_Atlas_v2.nc'.format(basedir)
na = 3 # Number of Annual Harmonics
BASETIME = np.datetime64('1990-01-01 00:00:00')

clim = sshdriver.load_ssh_clim(climfile)
amp = sshdriver.load_ssh_clim(ampfile)

clim, amp

(<xarray.Dataset>
 Dimensions:  (Nc: 225368, Nk: 80, Np: 454016, numsides: 8, time: 23)
 Coordinates:
     xv       (Nc) float64 ...
     yv       (Nc) float64 ...
     Nk       (Nc) int32 ...
   * time     (time) datetime64[ns] 2013-07-15T23:30:30 ... 2014-06-10T23:30:00
 Dimensions without coordinates: Nc, Np, numsides
 Data variables:
     cells    (Nc, numsides) int32 3058 3061 3060 2801 ... 316184 316187 -1 -1
     xp       (Np) float64 1.394e+07 1.271e+07 1.517e+07 ... 1.303e+07 1.303e+07
     yp       (Np) float64 -1.619e+06 -2.553e+06 ... -2.056e+06 -2.055e+06
     nfaces   (Nc) int32 ...
     dv       (Nc) float64 ...
     dz       (Nk) float64 ...
     eta      (time, Nc) float64 ...
     uc       (time, Nk, Nc) float64 ...
     vc       (time, Nk, Nc) float64 ...
     rho      (time, Nk, Nc) float64 ...
 Attributes:
     Description:  SUNTANS History file
     Author:       
     Created:      2020-08-21T12:50:55.346047
     Title:        SUNTANS climatology output, <xarray.

In [4]:
# Vertical grid is not stored in the climatology file so we will calculate it here
dz = clim._ds['dz'].values
nz = dz.shape[0]
dzw = np.zeros((nz+1,))
dzw[1::] = dz
z_w = np.cumsum(dzw)
z_r = 0.5*(z_w[0:-1] + z_w[1::])
z_r

array([4.11208050e+00, 1.25212850e+01, 2.13089035e+01, 3.04919650e+01,
       4.00882645e+01, 5.01163975e+01, 6.05957965e+01, 7.15467685e+01,
       8.29905340e+01, 9.49492690e+01, 1.07446147e+02, 1.20505385e+02,
       1.34152289e+02, 1.48413303e+02, 1.63316062e+02, 1.78889446e+02,
       1.95163632e+02, 2.12170157e+02, 2.29941975e+02, 2.48513524e+02,
       2.67920794e+02, 2.88201391e+02, 3.09394615e+02, 3.31541534e+02,
       3.54685063e+02, 3.78870052e+02, 4.04143366e+02, 4.30553978e+02,
       4.58153069e+02, 4.86994118e+02, 5.17133014e+02, 5.48628160e+02,
       5.81540589e+02, 6.15934076e+02, 6.51875272e+02, 6.89433820e+02,
       7.28682503e+02, 7.69697377e+02, 8.12557920e+02, 8.57347187e+02,
       9.04151972e+02, 9.53062972e+02, 1.00417497e+03, 1.05758700e+03,
       1.11340258e+03, 1.17172985e+03, 1.23268186e+03, 1.29637670e+03,
       1.36293782e+03, 1.43249418e+03, 1.50518058e+03, 1.58113787e+03,
       1.66051323e+03, 1.74346049e+03, 1.83014037e+03, 1.92072085e+03,
      

In [5]:
# Load the density data
Nk = clim._ds.Nk.values
rhoall = clim._ds['rho'].values[:]
dv = clim._ds['dv'].values[:]

In [6]:
def double_gaussian_N2_v2(z, beta):
    w1 = beta[6]
    w2 = 1-w1
    return beta[0,...] + beta[1,...] * (w1*np.exp(- ((z+beta[2,...])/beta[3,...])**2 )  +\
              w2*np.exp(-((z+beta[4,...])/beta[5,...])**2 ) )

In [ ]:
# Initialise the output variables
nt, nz, Nc = rhoall.shape
nparams = 7

N2_params = np.zeros((nt,nparams,Nc))
N2_err = np.zeros((nt, Nc))

RHO0 = 1024
GRAV = 9.81
cff1 = -GRAV/RHO0

for ii in tqdm(range(Nc)):
    initguess = [1e-5,  1e-4, 4, 2, 6.5, 2,0.5]
    # Skip shallow water
    if dv[ii]<100:
        continue
        
    nk = Nk[ii] 
    z = z_r[0:nk]
    zpr = -np.log(z)
    
    for tt in range(nt):
        bounds = [ (1e-6, 1e-5, 1, 0.1, 1.4, 0.1,0.05), (1e-1, 1e-1, 6, 6, 8, 6,0.95)]
        
        rho = rhoall[tt,0:nk,ii]
        N2 = cff1*grad_z(rho,-z)
        
        N2fit, f0, err = strat.fit_rho_lsq(N2, zpr, double_gaussian_N2_v2, bounds, initguess)
        
        # Use the last time-step as
        #initguess = f0
        
        N2_params[tt,:,ii] = f0
        N2_err[tt,ii] = err
    

  0%|          | 350/225368 [01:27<15:40:58,  3.99it/s]

In [ ]:
# plt.figure()
# plt.plot(zpr, N2,'.')
# plt.plot(zpr, N2fit)
# N2_params[tt,:,ii]

In [ ]:
# Test a harmonic fit
tsec = (clim._ds.time.values - BASETIME).astype(float)*1e-9

na = 4
aa, Aa, Ba, frq_all = strat.seasonal_harmonic_fit(N2_params, tsec, na)

In [ ]:
texstr = r"N^2(z) = \beta_0 + \beta_1  \left( \beta_6* \exp \left[- \left( \frac{z+\beta_2}{\beta_3} \right)^2 \right] + (1-\beta_6)*\exp \left[- \left( \frac{z+\beta_4}{\beta_5} \right)^2 \right] \right)"


In [ ]:
# Create an output dataset like the input amplitude data set
new_ds = amp._ds.copy()

# Drop a few variables
new_ds = new_ds.drop(labels=['amp_b_re','amp_b_im','alpha_n','cn','omega','N2'])
new_ds

In [ ]:
# Update some attributes
new_ds.attrs['Created'] = str(datetime.now())
new_ds.attrs['Title'] = 'SUNTANS density stratification climatology'
new_ds.attrs['Author'] = 'Matt Rayson (matt.rayson@uwa.edu.au)'
new_ds.attrs.update({'density_func':'double_gaussian_N2_v2'})
new_ds.attrs.update({'density_func_tex':texstr})
new_ds.attrs.update({'Number_Annual_Harmonics':na})#
new_ds.attrs.update({'ReferenceDate':BASETIME.astype(str)})

#ReferenceDate
new_ds.attrs


In [ ]:
# Convert the N2_params array to a DataArray
params = range(nparams)
omega = frq_all
dims = ('time','Nparams','Nc')
# coords = {'time':new_ds.time.values, 'Nparams':params, 'xv':range(Nc)}
ds_N2 = xr.DataArray(N2_params, dims=dims, attrs={'long_name':'N2 fit parameters'})
ds_omega = xr.DataArray(frq_all, dims=('Ntide',) )
ds_params = xr.DataArray(params, dims=('Nparams',) )

dims = ('Nparams','Nc')
ds_N2_mu = xr.DataArray(aa, dims=dims, attrs={'long_name':'N2 fit mean parameters'})
dims = ('Ntide','Nparams','Nc')
ds_N2_re = xr.DataArray(Aa, dims=dims, attrs={'long_name':'N2 fit real harmonic parameters'})
ds_N2_im = xr.DataArray(Ba, dims=dims, attrs={'long_name':'N2 fit imaginary harmonic parameters'})


ds2 = xr.Dataset({'N2_t':ds_N2,'omega':ds_omega,'params':ds_params,
                 'N2_mu':ds_N2_mu,'N2_re':ds_N2_re,'N2_im':ds_N2_im}).set_coords(['omega','params'])
ds2

In [ ]:
compflags = {'zlib':True, 'complevel':5}
encoding = {'N2_t':compflags, 'N2_mu':compflags, 'N2_re':compflags,'N2_im':compflags,}

In [ ]:
new_ds.merge(ds2).to_netcdf(N2file, encoding=encoding)

In [ ]:
#ds2.to_netcdf?

In [ ]:
xr.open_dataset(N2file)